# **THIS NOTEBOOK COMPUTES THE GROUND STATES OF V(x) = |x| AND OF V(x) = x^2 + a*exp(-bx^2)**
---
---
---
---
---

# IMPORTS AND GLOBAL VARIABLES

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

import keras
from keras import layers, models, optimizers

from scipy import integrate
from scipy import interpolate
from scipy.stats import norm

import time

# V(x) = |x|

In [ ]:
#################
#SECTION : GLOBAL VARIABLES
#################
hbar = 1
omega = 1
m = 1
a = -5.
b = 5.








#################
#SECTION : FUNCTION DEFINITION
#################


#W.F. stands for "wave function"


def energy_compute_harmonic (abscissa,psi):
  #interpolations
  tck_true = interpolate.splrep(abscissa, psi, k=3, s=0)                                         #W.F.
  #tck_true_carre = interpolate.splrep(abscissa, psi*psi, k=3, s=0)                               #W.F. squared
  tck_true_x = interpolate.splrep(abscissa, abscissa*abscissa*psi*psi, k=3, s=0)                 #W.F. squared*x^2
  der_true = interpolate.splev(abscissa, tck_true, der=1)                                        #W.F. derivative
  tck_true_der = interpolate.splrep(abscissa,der_true*der_true, k=3,s=0)                         #W.F. derivative spline 1000
  #int_true_carre = interpolate.splint(a,b,tck_true_carre)                                        #integral of W.F. squared
  int_true_x = interpolate.splint(a,b,tck_true_x)                                                #integral of W.F. squared*|x|
  int_true_der = interpolate.splint(a,b,tck_true_der)                                            #integral of derivative squared
  #energy
  Energy = ((-pow(hbar,2)/(2*m))*(psi[-1]*der_true[-1]-psi[0]*der_true[0] 
                             - int_true_der) + 0.5*m*omega*int_true_x ) #/ int_true_carre
  return Energy




def energy_compute_abs (abscissa,psi):
  #taking the absolute value of x as the potential : V(x) = |x|
  potential = np.abs(abscissa)
  #interpolations
  tck_true = interpolate.splrep(abscissa, psi, k=3, s=0)                                         #W.F.
  tck_true_carre = interpolate.splrep(abscissa, psi*psi, k=3, s=0)                               #W.F. squared
  tck_true_x = interpolate.splrep(abscissa, potential*psi*psi, k=3, s=0)                         #W.F. squared*|x|
  der_true = interpolate.splev(abscissa, tck_true, der=1)                                        #W.F. derivative
  tck_true_der = interpolate.splrep(abscissa,der_true*der_true, k=3,s=0)                         #W.F. derivative spline 1000
  int_true_carre = interpolate.splint(a,b,tck_true_carre)                                        #integral of W.F. squared
  int_true_x = interpolate.splint(a,b,tck_true_x)                                                #integral of W.F. squared*x^2 (<x^2>)
  int_true_der = interpolate.splint(a,b,tck_true_der)                                            #integral of derivative squared
  #energy
  Energy = ((-pow(hbar,2)/(2*m))*(psi[-1]*der_true[-1]-psi[0]*der_true[0] 
                             - int_true_der) + int_true_x ) / int_true_carre
  return Energy


def normalization (abscissa,psi):
  tck_true_carre = interpolate.splrep(abscissa, psi*psi, s=0)                       #W.F. squared
  int_true_carre = interpolate.splint(a,b,tck_true_carre)                           #integral of W.F. squared
  psi = psi*pow(1/int_true_carre,1/2)                                               #new normalized function

  return psi












#################
#SECTION : MINIMIZATION
#################

#Number of points on X and Y axis
pts = 200
# X axis
linx = np.linspace(a,b,pts)






#gaussian curve to compute the ground state of V(x) = |x|
wave2 = norm.pdf(linx)
#normalized
wave2 = normalization (linx,wave2)
#its energy
energy_wave2 = energy_compute_abs(linx,wave2)

#copy for plot at the end
first_target = wave2







# V(x) = |x|
#INITIALIZATION OF NEURAL NETWORK
fits = 7000 #how many iterations we want
model = models.Sequential([
  layers.Dense(100, input_shape=(1,), activation='relu'),
  layers.Dense(100, input_shape=(1,), activation='relu'),
  layers.Dense(100, input_shape=(1,), activation='relu'),
  layers.Dense(100, input_shape=(1,), activation='relu'),
  layers.Dense(1), # no activation -> linear function of the input
])
model.summary()
opt = optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse',optimizer=opt)
for i in range(0,fits):
  #one training
  model.fit(linx,wave2,epochs=1,batch_size=50,verbose=0)
  #prediction after one training
  predictions = model.predict(linx)
  #prediction is made positive, symetric and normalized
  preds = np.abs(predictions.reshape(-1))
  for j in range(0,pts):
    preds[j] = (preds[j]+preds[pts-1-j])/2
    preds[pts-1-j] = preds[j]
  preds = normalization(linx,preds)
  #energy of prediction
  energy_preds = energy_compute_abs(linx,preds)
  #we choose the function with the lowest energy
  if (energy_preds < energy_wave2):
    wave2 = preds
    energy_wave2 = energy_preds
    print('fit #',i+1)
    print('Energy = ',energy_preds)
  #go back to one training

#clearing the network at the end of the minimization
keras.backend.clear_session()



print('')
print('Energy found for |x| = ',energy_wave2)


plt.xlabel('x')
plt.ylabel('y')
plt.plot(linx,np.abs(linx),c='k',label = 'potential')
plt.plot(linx,first_target,c='darkgrey',label = 'first target (gauss. curve)')
plt.plot(linx,wave2,c='b',label = 'end result')
plt.ylim(-0.04,1)
plt.legend(title = 'minimiz. of energy for |x|',loc = 'lower right')
plt.savefig('minimization_01.pdf')
plt.savefig('minimization_01.png')

# V(x) = x^2 + a*exp(-bx^2)

In [ ]:
#################
#SECTION : FUNCTION DEFINITION
#################


#W.F. stands for "wave function"
#pot stands for "potential"




def energy_compute_double_harmonic (abscissa,psi,pot):
  #interpolations
  tck_true = interpolate.splrep(abscissa, psi, k=3, s=0)                                         #W.F.
  tck_true_pot = interpolate.splrep(abscissa, pot*psi*psi, k=3, s=0)                             #W.F. squared*potential
  der_true = interpolate.splev(abscissa, tck_true, der=1)                                        #W.F. derivative
  tck_true_der = interpolate.splrep(abscissa,der_true*der_true, k=3,s=0)                         #W.F. derivative spline 1000
  int_true_pot = interpolate.splint(a,b,tck_true_pot)                                            #integral of W.F. squared*potential
  int_true_der = interpolate.splint(a,b,tck_true_der)                                            #integral of derivative squared
  #energy
  Energy = ((-pow(hbar,2)/(2*m))*(psi[-1]*der_true[-1]-psi[0]*der_true[0] 
                             - int_true_der) + int_true_pot )
  return Energy


def normalization (abscissa,psi):
  tck_true_carre = interpolate.splrep(abscissa, psi*psi, s=0)                       #W.F. squared
  int_true_carre = interpolate.splint(a,b,tck_true_carre)                           #integral of W.F. squared
  psi = psi*pow(1/int_true_carre,1/2)                                               #new normalized function
  return psi











#################
#SECTION : POTENTIAL COMPUTATION
#################
#Number of points on X and Y axis
pts = 200
# X axis
linx = np.linspace(a,b,pts)

x = a
h = 10/pts

#computing a gaussian potential
potential = np.zeros_like(linx, dtype=float)
for i in range(0,pts):
  potential[i] = 3*math.exp(-4*pow(x,2))
  x+=h
#computing V(x) = harmonic + gaussian
potential = (1/2)*m*omega*omega*linx*linx + potential
#V(x) is made more symmetric (tends not to be when pts is ~ 100)
for j in range(0,pts):
    potential[j] = (potential[j]+potential[pts-1-j])/2
    potential[pts-1-j] = potential[j]







#################
#SECTION : MINIMIZATION
#################




#gaussian curve to compute the ground state of V(x) = x*x
wave = norm.pdf(linx)
#normalized
wave = normalization (linx,wave)
#its energy
energy_wave = energy_compute_double_harmonic(linx,wave,potential)
#copy for plot at the end
first_target = wave







# V(x) = x*x + a*exp(-b*x^2)
#INITIALIZATION OF NEURAL NETWORK
fits = 7000 #how many iterations we want
model = models.Sequential([
  layers.Dense(100, input_shape=(1,), activation='relu'),
  layers.Dense(100, input_shape=(1,), activation='relu'),
  layers.Dense(100, input_shape=(1,), activation='relu'),
  layers.Dense(100, input_shape=(1,), activation='relu'),
  layers.Dense(1), # no activation -> linear function of the input
])
model.summary()
opt = optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse',optimizer=opt)
for i in range(0,fits):
  #one training
  model.fit(linx,wave,epochs=1,batch_size=50,verbose=0)
  #prediction after one training
  predictions = model.predict(linx)
  #prediction is made positive, symetric and normalized
  preds = np.abs(predictions.reshape(-1))
  for j in range(0,pts):
    preds[j] = (preds[j]+preds[pts-1-j])/2
    preds[pts-1-j] = preds[j]
  preds = normalization(linx,preds)
  #energy of prediction
  energy_preds = energy_compute_double_harmonic(linx,preds,potential)
  #we choose the function with the lowest energy
  if (energy_preds < energy_wave):
    wave = preds
    energy_wave = energy_preds
    print('fit #',i+1)
    print('Energy = ',energy_preds)
  #go back to one training

#clearing the network at the end of the minimization
keras.backend.clear_session()



print('')
print('Energy found = ',energy_wave)


plt.xlabel('x')
plt.ylabel('y')
plt.plot(linx,potential,c='k',label = 'potential')
plt.plot(linx,first_target,c='darkgrey',label = 'first target (gauss. curve)')
plt.plot(linx,wave,c='b',label = 'end result')
plt.ylim(-0.04,3.5)
plt.legend(title = 'minimiz. of energy for ammonia mol.',loc = 'center right')
plt.savefig('minimization_01.pdf')
plt.savefig('minimization_01.png')